In [242]:
# Based on paper 
# Herrera, Kannala, Heikkilä - Forget the checkerboard: practical self-calibration using a planar scene

import numpy as np
import math 

def rotationsFromRotationMatrix(R):
    return np.vstack((0,0,0))

def rotationMatrixFromRotations(rx,ry,rz):
    Rx = np.matrix([[1, 0, 0],
                [0, math.cos(rx), -math.sin(rx)],
                [0, math.sin(rx), math.cos(rx)]
                ])

    Ry = np.matrix([[math.cos(ry), 0, math.sin(ry)],
                    [0, 1, 0],
                    [-math.sin(ry), 0, math.cos(ry)  ]
                    ])

    Rz = np.matrix([[math.cos(rz), -math.sin(rz), 0],
                    [math.sin(rz), math.cos(rz), 0],
                    [0, 0, 1]
                    ])
    R = np.dot(Rz, np.dot( Ry, Rx ))
    return R

def lookAtRotationMatrix(C,x,y,z):
    # Unit vector from  camera to lookat-point
    uz = np.array([x,y,z])-np.squeeze(C)
    uz = uz/np.linalg.norm(uz)

    # Camera is approx uprights
    uy = np.array([0,0,1])
    
    ux = np.cross(uy,uz)
    ux = ux/np.linalg.norm(ux)

    # Calculate correct y
    uy = np.cross(uz,ux)
    
    return np.vstack((ux,uy,uz))

class Camera:
    def __init__(self):
        self.radial_distortion = np.array([0.1, -0.01])
        self.p0 = np.matrix([320,240]).transpose()
        self.f = np.array([600, 600])
        
        self.K = np.matrix([[self.f[0],0,self.p0[0]],
                           [0,self.f[1],self.p0[1]],
                          [0,0,1]])
        
        # dist distance from origin
        dist = 2000
        self.C = np.matrix(np.random.rand(3)).transpose()
        self.C = self.C/np.linalg.norm(self.C)*dist
        
        self.R = lookAtRotationMatrix(self.C,x=0,y=0,z=0)
        self.P = self.K*np.hstack((self.R,-self.R*self.C))

    def transform(self,x,y,z):
        # To homogenous coordinates
        X = np.vstack((x,y,z,np.ones(len(x))))
        
        # Project
        X_ = self.P*X
        
        # Set homogenous coordinate to 1
        xp = X_[0:2,:]/X_[2,:]
        
        ## Calculate distance to the optical center of the image
        # Substract center of distrotion
        d = xp-self.p0
        # Calculate distance to origin
        tmp = np.multiply(d,d)
        r2 = tmp[0,:]+tmp[1,:]
        r4 = np.square(r2)
        
        # Amount of distortion
        distortion_multiplier = 1 + r2*self.radial_distortion[0] + r4*self.radial_distortion[1]
        
        # Multiply "centered coordinates" with distortion and add distortion center back
        distorted = d*distortion_multiplier + self.p0
        
        return distorted
        
cam = Camera()
cam.transform([30],[0],[0])

matrix([[290.9756144 ],
        [311.52402047]])